<a href="https://colab.research.google.com/github/cbonnin88/RailFlow/blob/main/Reverse_ETL_for_CRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import json
from google.cloud import bigquery
from google.colab import auth

In [8]:
auth.authenticate_user()

In [25]:
# 1. Setup Client
project_id = 'railflow-484310'
client = bigquery.Client(project=project_id, location='europe-west9')

In [26]:
# 2. The Marketing Query
# Find users who searched in the last 7 days but have ZERO bookings
query = """
WITH recent_searches AS (
  SELECT DISTINCT
    user_id
  FROM `dbt_railflow_dev.int_search_bookings`
  WHERE search_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
),
bookers AS (
  SELECT DISTINCT user_id
  FROM `dbt_railflow_dev.int_search_bookings`
  WHERE is_converted = TRUE
)

SELECT
  r.user_id,
  'high_churn_risk' AS segment,
  'send_winback_email'  AS next_action
FROM recent_searches AS r
LEFT JOIN bookers AS b
  ON r.user_id = b.user_id
WHERE b.user_id IS NULL
LIMIT 5
"""

In [27]:
df_churn_risk = client.query(query).to_dataframe()

In [28]:
print(f"📉 Found {len(df_churn_risk)} users at risk of churning.")

📉 Found 0 users at risk of churning.


In [29]:
# 4. Generate JSON payload (Simulating a sync to Braze/Salesforce)
# In real life, this would be a POST to an API endpoint

marketing_payload = df_churn_risk.to_json(orient='records',indent=2)

In [30]:
# Save to file (Simulating the export)
with open('audience_export_braze.json','w') as f:
  f.write(marketing_payload)

In [31]:
print("✅ Audience exported to 'audience_export_braze.json'. content preview:")
print(marketing_payload)

✅ Audience exported to 'audience_export_braze.json'. content preview:
[

]
